# Testing in Python

Der Kurs zum Thema Testing in Python ist folgendermaßen gegliedert:
1. Struktur
2. Implementierung der Tests mit `pytest`
3. Hilfreiche Funktionen
4. Test Coverage
5. Aufgaben

* Tests werden normalerweise nicht in Jupyter Notebooks definiert und ausgeführt werden. Um in Jupyter Notebooks trotzdem das Modul `pytest` verwenden und demonstrieren zu können, nutzen wir daher im Folgenden das Modul `IPytest`. Für das Module `coverage` existiert eine solche Lösung in Jupyter Notebook nicht. Code Coverage Tests führen wir daher exemplarisch über eine `Shell` aus.
* alle Funktionen und Tests dieses Tutorials befinden sich auch als Python-Dateien in diesem git. <br> Öffne hierzu `PyCharm` --> `Check out from Version Control` --> `git` und gib den `URL`-Link ein.
* Öffne anschließend `Settings` --> `Tools` --> `Python Integrated Tools` und wähle als `Default test runner` `pytest`. Klicke `OK` um die Einstellung zu speichern.

## 1. Struktur

<img src="attachment:grafik.png" width="700">

* Hauptskript, das alle wichtigen Funktionsaufrufe enthält (`mainFunction.py`)
* Skript mit allen nötigen Funktionsdefinitionen (`functions.py`)
* Skript mit allen benötigten Tests für functions.R (`tests.py`)
* Optionaler Ordner mit kleinen Testdatensätzen (Testdaten)<br>
<br>
* Ein Testskript für jedes Skript mit Funktionsdefinitionen (Konzept Unittesting)

## 2. Implementierung

* Alle relevanten Pakete müssen geladen werden
* Skript mit allen Funktionen muss geladen werden
* Body enthält alle Tests für das Testskript `Functions.py`
* Alphabetisch geordnet
* Optional: gruppiert nach Codeblöcken (Preprocessing, Forecast, …)

Informationen zur Installation von `pytest` findest Du hier: https://docs.pytest.org/en/latest/getting-started.html

* `pytests` ermöglichen einfaches und schnelles Testing
* wenig Codezeilen benötigt
* Alternative: Framework `unittest`<br>
<br>
* schreibe zusammengehörige Tests in eine Funktion mit `test_>>function name<<_>>short description<<():`
* die kurze Beschreibung liefert eine Erklärung, was getestet wird (z.B. `happypath`, `extremepath`)
* schreibe als Kommentar dahinter, was der Test macht. Dies hilft beim Debugging von fehlerhaften Tests. Hilfreich sind dort auch Hinweise, ob es sich z.B. um einen Normaltest oder einen Extremtest handelt.
* Funktion für einen Test ist `assert`

In [3]:
def multiply_by_2(x):
    return x*2

<b>Führe Tests in dem Notebook aus<b>

In [4]:
import ipytest
import pytest
ipytest.autoconfig()

#1. Möglichkeit
def test_multiply_by_2_happypath(): #test for correct execution (different values)
    assert multiply_by_2(0) == 0
    assert multiply_by_2(1) == 2
    assert multiply_by_2(2) == 4
    assert multiply_by_2(3) == 6
    
#2. Möglichkeit
@pytest.mark.parametrize('input,expected', [ #schreibe in Anführungszeichen die Variablennamen
    (-1, -2),
    (-3, -6),
    (4, 8),
    (5, 10)
])
def test_multiply_by_2_parametrized_happypath(input, expected): #test for correct execution (different values)
    assert multiply_by_2(input) == expected,"test failed because "+str(input)+"*2/="+str(expected)

In [5]:
ipytest.run('-qq')

.....                                                                    [100%]


<b> Führe Tests in einem anderen Notebook aus<b>

* Öffne hierzu `PyCharm` wie oben beschrieben und führe die `Shel1 1` aus.
* `Shell 1` ist ein `PowerShell`-Skript, Hinweise zur Ausführung unter: https://www.heise.de/tipps-tricks/Windows-Powershell-Skript-ausfuehren-4672163.html

## 3. Hilfreiche Funktionen

### Dateipfad ausgeben

* Die Funktion `.getcwd()` aus dem Paket `os` gibt den Pfad des Projekts zurück.
* benutzte keine hart codierten Pfade, auch nicht in Tests
* Es hat sich herausgestellt, dass dadurch wesentlich leichter von verschiedenen Benutzern Tests erstellt und geprüft werden können.
* Die Pfade in den Tests einfach ausgehend vom Projektpfad mit `os.getcwd()` angeben.

In [6]:
import os
os.getcwd()

'/Users/udis/Documents/Werkstudentenjob/Python_Tutorial/Testing'

### Handling Errors

* ermöglicht, gewisse Fehlermeldungen bei Tests abzufangen
* Falls innerhalb einer Funktion 2-mal Fehler mit selber Klasse geprüft werden, muss zusätzlich auf die Fehlermeldung wie bei der 2. Möglichkeit geprüft werden, um genau bestimmen zu können, wo der Abbruch stattfindet.

In [7]:
%%run_pytest[clean] -qq 

def test_multiply_by_2_extremepath(): #test for correct execution (expecting error)
    with pytest.raises(ZeroDivisionError):#erwartet, dass ein ZeroDivisionError geworfen wird; passiert das nicht wird
        1 / multiply_by_2(0)                             #ein Fehler geworfen

.                                                                        [100%]


### `Fixture`

* Das Ziel von `Fixtures` ist, eine fixe Startlinie zu definieren, wodurch Tests zuverlässig ausgeführt werden können.
* `pytest` `Fixtures` stellen Daten zur Verfügung und bilden das Setup für deine Tests.
* Ausführlichere Angaben zu `Fixture` findest Du hier: https://docs.pytest.org/en/2.9.1/fixture.html

In [28]:
import pandas as pd

@pytest.fixture
def fixture_example():
    datapath = os.getcwd() +"/Aufgaben/test/Unit/data/Abverkauf.csv"
    return pd.read_csv(datapath, delimiter = ";", encoding='latin-1') 

In [30]:
%%run_pytest[clean] -qq 

def test_multiply_by_2_happypath_2(fixture_example): #test for correct execution
    assert fixture_example["Material"][0] == 9148

.                                                                        [100%]


### Debugging

* nutzbar in Entwicklungsumgebungen wie z.B. `PyCharm`, `Eclipse`, ...
* Schlagen Tests fehl, verwendet man den Debugger mit entsprechend gesetzten Break Points, um die Ausführung Schritt für Schritt durchzugehen.

<img src="attachment:image.png" width="700">

* Der rote Punkt ist ein Break Point. Betätigt man die grüne Fliege/Bug im rechten oberen Bildrand, startet der Debugger und der Interpreter stoppt, sobald der Break Point erreicht wird.
* Nun lassen sich die Werte der Parameter ansehen (links unten im Bild) und die Ausführung zeilenweise durchgehen.

## 4. Test Coverage

* Installiere folgende Module
    * Coverage: *pip install coverage*, *pip install pytest-cov*
    * `HTML`-Darstellung mit *pip install pytest-html*
* Führe nun `Shell 2` aus

<b>Einfache Version:</b><br>
<blockquote>
    coverage run -m pytest arg1 arg2 arg3$\;\;\;$ führt Tests aus<br>
    coverage report -i $\;\;\;\;\;\;$ $\;\;\;\;\;\;$ $\;\;\;\;\;\;$  $\;\;\;\;\;\;$ liefert Ergebnisse im Fenster<br>
    coverage html $\;\;\;\;\;$ $\;\;\;\;\;\;$ $\;\;\;\;\;\;$$\;\;\;\;\;\;$$\;\;\;\;\;\;$ liefert ausführliche Ergebnisse im HTML-Format</blockquote><br>
<b>Speichern der Ergebnisse in XML-Dateien:</b><br>
<blockquote>python3 -m pytest -v --cov-report xml:target_dir/py_cov.xml --cov=algorithm_implementation --junitxml=target_dir/py-test-results.xml ./tests<br></blockquote>
<b>Speichern der Ergebnisse in HTML-Dateien:</b><br>
<blockquote>python3 -m pytest -v --cov-report html:target_dir/py_cov.html --cov=algorithm_implementation --html=target_dir/py-test-results.html ./tests.py<br></blockquote>
<b>Nur Coverage:</b><br>
<blockquote>python3 -m pytest -v --cov-report xml:file</blockquote>

* Test Coverage misst den Prozentsatz der Zeilen des Skripts `functions.py`, die durch die Tests abgedeckt sind.
* Test Coverage ist ein notwendiges Kriterium für qualitativ hochwertigen Code, jedoch kein hinreichendes Kriterium
* Ziel ist ein möglichst hoher Test Coverage durch <b>sinnvolle</b> Tests.

In [10]:
def absolute_value(x):
    if x>=0:
        return x
    else:
        return -x

In [13]:
%%run_pytest[clean] -qq 

def test_absolute_value_happypath_positive(): #test for correct excecution (positive values)
    assert absolute_value(0) == 0
    assert absolute_value(1) == 1
    assert absolute_value(2) == 2
    assert absolute_value(3) == 3
    
def test_absolute_value_happypath_negative(): #test correct excecution (negative values)
    assert absolute_value(0) == 0
    assert absolute_value(-1) == 1
    assert absolute_value(2) == 2
    assert absolute_value(-3) == 3

..                                                                       [100%]


* die Funktion besitzt 4 Zeilen Code 
* beim ersten Test würde man eine Test Coverage von 50% erhalten, da keine negativen Werte getest werden
* beim zweiten Test erhält man eine Test Coverage von 100%, da sowohl positive als auch negative Werte getestet werden.

Mehr Informationen zu `pytest` findest Du hier:
* https://docs.pytest.org/en/latest/talks.html
* https://docs.pytest.org/en/2.8.7/fixture.html
* https://docs.pytest.org/en/latest/xunit_setup.html

## 5. Aufgaben

Die Aufgaben befinden sich in der `PDF`-Datei *aufgaben_testing_in_python* im Ordner *Aufgaben*, die dazugehörigen Daten im Ordner *data*. 